# Asses vegetation in the city of Munich (work in progress)

Inspired by: https://www.youtube.com/watch?v=gi4UdFsayoM

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

### The Workflow

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest)
- import multiple Landsat images in summer (for highest vegetation)
- determine which images are best for analysis
- create various NDVI map outputs to assess the vegetation

## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


## Import libraries

In [67]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

## Authenticate Earth Engine

In [68]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Check setup (should print "USGS Landsat 8 Level 2, Collection 2, Tier 1")
print(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").get("title").getInfo())

USGS Landsat 8 Level 2, Collection 2, Tier 1


For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [69]:
# coordinates of München
lat = 48.13  #39.444012
lon = 11.56  #-121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
# clouds impact quality, so we need to chose between a variety of images
start_date = '2022-05-01'

# end date
end_date = '2022-08-31'

## Get Landsat 8 data

In [70]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [71]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 17


In [1]:
# information about the first image in our collection
landsat.first().getInfo()

In [73]:
# what about cloud cover of our first image?
# Number is in percentage
landsat.first().get('CLOUD_COVER').getInfo()

55.99

In [74]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2022-06-25'

## Bands

[Landsat 8 Bands](https://www.usgs.gov/media/images/landsat-8-band-designations)

In [75]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [76]:
# put the images in a list
landsat_list = landsat.toList(landsat.size())

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine) <-- Read! (Not so easy to set, roughly speaking, they control brightness)

In [77]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [78]:
landsat.size().getInfo()

17

In [79]:
# create an empty data container
data = []

# loop through each image and display it
# `landsat.size().getInfo()` gives number of images in list
for i in range(landsat.size().getInfo()):
    # singe Image-object
    image = ee.Image(landsat_list.get(i))
    
    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = image.get('CLOUD_COVER').getInfo()

    if cloud < 3.:
        # print the image info
        print('Image #', i, date, 'Cloud cover:', cloud)
        
        # display the image
        # display(Image(url = image.getThumbUrl(parameters)))
    
        # data to list
        this_data = [i, date, cloud]
    
        # append the data 
        data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 4 2022-05-15 Cloud cover: 2.4
Image # 5 2022-05-31 Cloud cover: 3.69
Image # 7 2022-07-02 Cloud cover: 3.22
Image # 8 2022-07-18 Cloud cover: 0.3
Image # 9 2022-08-03 Cloud cover: 2.01
Image # 13 2022-07-02 Cloud cover: 1.88
Image # 14 2022-07-18 Cloud cover: 1.94
Image # 15 2022-08-03 Cloud cover: 2.61


In [80]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,4,2022-05-15,2.40
1,5,2022-05-31,3.69
2,7,2022-07-02,3.22
3,8,2022-07-18,0.30
4,9,2022-08-03,2.01
5,13,2022-07-02,1.88
6,14,2022-07-18,1.94
7,15,2022-08-03,2.61


## Selecting images, zooming in

Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study, ie. the image having the lowest cloud cover.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [98]:
# create a list of images we want (before, during, after)
landsat_sequence = [8]

In [99]:
# Define a region of interest with a buffer zone of 3 km
# Initially, the POI is for selecting which images, the ROI controls which radius around the point is displayed, ie. the zoom
roi = poi.buffer(3000) # meters

In [100]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region': roi
             }

In [101]:
for i in landsat_sequence:

    image = ee.Image(landsat_list.get(i))   
    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = image.get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = image.getThumbUrl(parameters)))

Image # 8 2022-07-18 Cloud cover: 0.3


## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/

### Calculating NDVI in Google Earth Engine

https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [102]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [103]:
for i in landsat_sequence:

    image = ee.Image(landsat_list.get(i))

    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #', i, date)
    
    # display the image

    display(Image(url=image.normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 8 2022-07-18


In [107]:
import requests

image = ee.Image(landsat_list.get(0))
# Multi-band GeoTIFF file.
url = image.normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)
response = requests.get(url)
with open('multi_band', 'wb') as fd:
  fd.write(response.content) 

## Folium

Of course, we can't end the lab without an interactive map. For this, we can use folium, which uses the open-source javascript mapping library leaflet.

    https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:

    https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

Currently, I am not able to add the cropped image of munich to the folium map.

In [87]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
# m  # Show the map

In [88]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [89]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

# TODO Add cropped images of Munich to folium

## Save the folium map as an html file

In [90]:
my_map.save('camp.html')

# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    